In [1]:
%load_ext autoreload

In [2]:
from torch_ham.models import ConditionalUNet

In [ ]:
model = ConditionalUNet(
    input_size = (128, 128),
    condition_size = (,
    kernel_size: Union[int, Tuple[int]],
    downsample: Union[int, Tuple[int]],
    depth: int,
    in_channels: int = 1,
    channel_scale: int = 1,
)